In [4]:
import os
import time
import shutil
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import keras
import gc
from PIL import Image
%matplotlib inline
from keras.models import Sequential,Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
!unzip /content/gdrive/MyDrive/Caltech256.zip

Streaming output truncated to the last 5000 lines.
  inflating: Caltech256/229.tricycle/229_0010.jpg  
  inflating: Caltech256/229.tricycle/229_0011.jpg  
  inflating: Caltech256/229.tricycle/229_0012.jpg  
  inflating: Caltech256/229.tricycle/229_0013.jpg  
  inflating: Caltech256/229.tricycle/229_0014.jpg  
  inflating: Caltech256/229.tricycle/229_0015.jpg  
  inflating: Caltech256/229.tricycle/229_0016.jpg  
  inflating: Caltech256/229.tricycle/229_0017.jpg  
  inflating: Caltech256/229.tricycle/229_0018.jpg  
  inflating: Caltech256/229.tricycle/229_0019.jpg  
  inflating: Caltech256/229.tricycle/229_0020.jpg  
  inflating: Caltech256/229.tricycle/229_0021.jpg  
  inflating: Caltech256/229.tricycle/229_0022.jpg  
  inflating: Caltech256/229.tricycle/229_0023.jpg  
  inflating: Caltech256/229.tricycle/229_0024.jpg  
  inflating: Caltech256/229.tricycle/229_0025.jpg  
  inflating: Caltech256/229.tricycle/229_0026.jpg  
  inflating: Caltech256/229.tricycle/229_0027.jpg  
  inflating: 

In [6]:
path = '/content/Caltech256/'
train_path = '/content/caltech/train/'
test_path = '/content/caltech/test/'

In [7]:
os.chdir(path)
folders = os.listdir()
img_size = 224

In [8]:
# splitting training and test data into different directories
os.chdir('/content/')
os.mkdir('caltech')
os.mkdir(train_path)
os.mkdir(test_path)
os.mkdir(test_path+str('predict/'))
for folder in range(len(folders)):
  folder_paths = path+str(folders[folder])+str('/')
  os.mkdir(os.path.join(train_path, folders[folder]))
  os.chdir(folder_paths)
  image_in_folder = os.listdir()
  count = 1
  for image in range(len(image_in_folder)):
    if count <= 30:
      shutil.move(image_in_folder[image], train_path+str('/')+folders[folder]+str('/'))
    else:
      shutil.move(image_in_folder[image], test_path+str('predict/'))
    count += 1

shutil.rmtree(path)

In [9]:
def train_model():

  epochs = 80
  lr = 0.0001
  batch_size = 32
  decay = lr/epochs
  adam = Adam(lr=lr, decay=decay)
  
  resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(img_size,img_size,3))
  output = resnet.layers[-1].output
  output = keras.layers.Flatten()(output)
  resnet = Model(resnet.input, output)
  
  model = Sequential()
  model.add(resnet)
  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dense(257, activation='softmax'))
  
  model.compile(adam, 
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  
  checkpoint_filepath = '/content/trainedmodel.h5'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                              save_weights_only=True,
                              monitor='accuracy',
                              mode='max',
                              save_best_only=True)
  
  model.summary()


  
  train_datagen = ImageDataGenerator(rescale=1./255)
        
  train_generator = train_datagen.flow_from_directory(
          train_path,
          target_size=(img_size, img_size),
          batch_size=batch_size,
          shuffle=True,
          class_mode='categorical')

  test_datagen = ImageDataGenerator(rescale=1./255)  
  test_generator = test_datagen.flow_from_directory(
          test_path,
          target_size=(img_size, img_size),
          batch_size=1,
          class_mode=None,
          shuffle=False)

  history = model.fit(train_generator, batch_size=batch_size, epochs=10, shuffle=True, callbacks=[model_checkpoint_callback],
                      steps_per_epoch=train_generator.n/batch_size)
  
  test_generator.reset()
  pred=model.predict_generator(test_generator, steps=test_generator.n,verbose=1)

  predicted_class_indices=np.argmax(pred,axis=1)
  labels = (train_generator.class_indices)
  labels = dict((v,k) for k,v in labels.items())
  
  predictions = [labels[k] for k in predicted_class_indices]

  filenames=test_generator.filenames
  
  # Calculate Testing accuracy
  targets = [x.split('.')[0] for x in predictions]
  filenames = [x.split('/')[1].split('_')[0] for x in filenames]

  accuracy = sum(1 for x,y in zip(filenames,targets) if x==y)
  accuracy = (accuracy / len(filenames))*100
  
  model.save_weights(checkpoint_filepath)
  return model, history, accuracy

In [10]:
                                                                  #Execution begins from here
training_accuracy = 0
testing_accuracy = 0
training_time  = 0
runs = 3
for i in range(runs):
  print("\n==========RUN " + str(i+1) + "==========\n")
  
  start_time = time.time()
  model, history, test_accuracy = train_model()
  end_time = time.time() - start_time

  print("\nTraining Accuracy = "+ str(history.history.get('accuracy')[-1] * 100) + "%")
  print("Testing Accuracy = " +  str(test_accuracy) + "%")
  print("Training time = " +  str(end_time) + " seconds")

  training_accuracy += history.history.get('accuracy')[-1] * 100
  testing_accuracy += test_accuracy
  training_time += end_time

  K.clear_session()
  del model
  gc.collect()
  time.sleep(10)
  
print("\nAverage training accuracy for 3 runs = " + str(training_accuracy/3)+"%")
print("Average testing accuracy for 3 runs = " + str(testing_accuracy/3)+"%")
print("Average training time for 3 runs = " + str(training_time/3)+" seconds")


==========RUN 1==========



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


94781440/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 100352)            23587712  
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 257)               131841    
                                                                 
Total params: 75,102,337
Trainable params: 75,048,193
Non

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


22897/22897 [==============================] - 600s 26ms/step

Training Accuracy = 100.0%
Testing Accuracy = 77.27649910468621%
Training time = 2348.9990038871765 seconds

==========RUN 2==========

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 100352)            23587712  
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 257)               